Cheminformatics Datasets 
https://github.com/GLambard/Molecules_Dataset_Collection/tree/master/latest

In [1]:
from dl_util import *
from ml_util import *

from sklearn.preprocessing import StandardScaler
from eval_class_util import *
from run_eval import split_fit_plot_predict

Using TensorFlow backend.


### Extracting SMILES and converting into sequence 

In [2]:
esol = pd.read_csv('eval_datasets/train.csv')
esol.head()

,File ID,SMILE,Energy
0,0.xyz,C[NH+]1CC1C(C)(C)O,-1879.273366
1,1.xyz,COC1CNCC1=O,-1598.701542
2,2.xyz,Nc1cccc(N)c1,-1633.722192
3,3.xyz,O=CC1COCO1,-1256.754945
4,4.xyz,N#CCOCCO,-1317.974723


In [3]:
esol_smiles = esol['SMILE'].tolist()
esol_solubility = esol['Energy'].tolist()

In [4]:
len_esol_smiles = []
for SMILE in esol_smiles:
    len_esol_smiles += [len(SMILE)]
    
max_len_esol = max(len_esol_smiles)

esol_vocab_size = generate_onehot_encoding(esol_smiles,'vocab_size')
esol_vocab_size, max_len_esol

(24, 34)

In [5]:
esol_encodings = generate_onehot_encoding(esol_smiles)
esol_sequences = sequence.pad_sequences(esol_encodings, maxlen=max_len_esol)

In [6]:
layers=2
optimizer ="adam"
model_type ="regression"
model = Sequential()
model.add(Embedding(esol_vocab_size,  32, input_length=max_len_esol))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
if layers==3:
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
# if optimizer == "adam" and lr!= 0.001:
#     print("Setting learning rate to"+str(lr))
#     optimizer = tf.train.AdamOptimizer(lr)

if model_type == "classification":
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['acc', precision,recall,auc])
else:
    model.add(Dense(1))
    model.compile(loss='mse',optimizer=optimizer, metrics=['mape'])
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 34, 32)            768       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 34, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 17, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 17, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 32)             0         
_____________________________________________

In [7]:
X1_train, X1_test, y_train, y_test = train_test_split(esol_sequences, esol_solubility, test_size=0.2, random_state=1024)

In [16]:
model.fit(X1_train, y_train, shuffle=True, validation_split=0.1,\
                        epochs=5, batch_size=4, verbose=1)

Train on 7200 samples, validate on 800 samples
Epoch 1/5
7200/7200 [==============================] - 2s - loss: 780.6718 - mean_absolute_percentage_error: 1.3819 - val_loss: 812.0225 - val_mean_absolute_percentage_error: 1.2716
Epoch 2/5
7200/7200 [==============================] - 2s - loss: 779.6038 - mean_absolute_percentage_error: 1.3763 - val_loss: 872.1462 - val_mean_absolute_percentage_error: 1.3106
Epoch 3/5
7200/7200 [==============================] - 2s - loss: 766.3839 - mean_absolute_percentage_error: 1.3692 - val_loss: 781.1762 - val_mean_absolute_percentage_error: 1.2508
Epoch 4/5
7200/7200 [==============================] - 2s - loss: 761.1423 - mean_absolute_percentage_error: 1.3728 - val_loss: 837.1888 - val_mean_absolute_percentage_error: 1.3321
Epoch 5/5
7200/7200 [==============================] - 2s - loss: 722.9485 - mean_absolute_percentage_error: 1.3441 - val_loss: 872.6360 - val_mean_absolute_percentage_error: 1.3614


In [17]:
y_predict = model.predict(X1_test).reshape(1,-1)[0]

In [18]:
y_test = np.array(y_test)

In [19]:
type(y_predict), type(y_test)

(numpy.ndarray, numpy.ndarray)

In [20]:
y_test[:5]

array([-1684.03398063, -1649.43878195, -1474.12720006, -1126.64722885,
       -1467.78308407])

In [21]:
import math
import sklearn

rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_test, y_predict))
print(rmse)

29.6750235199


In [22]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_predict)

20.755345523899145

In [ ]:
# sum(y_test)/len(y_test)

In [ ]:
# y_test,y_predict = split_fit_plot_predict(cnn_model,  esol_sequences, esol_maccs, esol_solubility_standard, esol_vocab_size, max_len_esol, "esol",\
                      batch_size=4, epochs=10)

In [ ]:
# for i in range(len(y_test)):
#     item = y_test[i]
#     if item == 0:
#         print(item,i)

In [ ]:
# for i in range(len(y_predict)):
#     item = y_predict[i]
#     if item == 0:
#         print(item,i)

## Predictions 

In [ ]:
# split_fit_plot_predict(cnn_model,  esol_sequences, esol_maccs, esol_solubility, esol_vocab_size, max_len_esol, "esol",\
#                       batch_size=4, epochs=40)